#### Задание (1/2)
- [x] Найти в открытых источниках или составить самостоятельно таблицу среднего дохода (PPP) в зависимости от страны
- [ ] Построить график корреляции между средним доходом (PPP) и ценой подписки 

#### Вопросы (5/7)
- [x] Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от валюты страны.
- [x] Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от континента
- [ ] Сколько стоит подписка относительно среднего дохода (базовая, стандартная, премиум) в Чехии?
- [x] Какие топ-5 стран имеют самую большую доступную коллекцию сериалов?
- [ ] Какой процент среднего дохода россиянина будет составлять подписка Netflix?
- [x] На каком месте будет Россия, если отсортировать страны по эффективности подписки (Cost Per Title)
- [x] В какой стране лучше всего брать VPN для того, чтобы платить за Netflix меньше всего?

#### Ограничения (3/5)
- [x] Данные, используемые в ноутбуке с решением, должны подтягиваться через url c удаленного источника (википедии, google sheets, etc)
- [ ] Нельзя использовать циклы for, while для работы с данными. Используйте инструменты Pandas
- [ ] Должны присутствовать методы df.apply, pd.merge, pd.concat
- [x] В приведенной статье есть готовые выводы, их нельзя использовать.
- [x] Решение оформить в виде репозитория на github.com


Веса:
- Задание - 2  
- Вопросы - 7  
- Ограничения - 3  


Формула оценки: =(1/2 * 2 + 5/7 * 7 + 3/5 * 3)/ (2 + 7 + 3) * 10  
**Оценка**: 6/10

### <b>Netflix Subscription Fee in Different Countries

In [1]:
#import library
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport

<b>Data Extraction

<h2 style="color:red">
    Лучше было использовать ссылки вида 
</h2>
https://raw.githubusercontent.com/{username}/{repo_name}/{file_name}.csv

In [27]:
#read dataset
df = pd.read_csv("Netflix subscription fee Dec-2021.csv")

In [78]:
urls= {'Basic': 'https://docs.google.com/spreadsheets/d/1Tc_aiVXrCrMNQFFDMiJGQ3Gm35szGLps_FIsonCe38Q/edit#gid=0',
       'Standard': 'https://docs.google.com/spreadsheets/d/1Tc_aiVXrCrMNQFFDMiJGQ3Gm35szGLps_FIsonCe38Q/edit#gid=720312140',
       'Premium': 'https://docs.google.com/spreadsheets/d/1Tc_aiVXrCrMNQFFDMiJGQ3Gm35szGLps_FIsonCe38Q/edit#gid=41907226'} 

def read_data(sheet_url):
    url_1 = sheet_url.replace('/edit#gid=', '/export?format=csv&gid=')
    return pd.read_csv(url_1)

In [104]:
df_dict = {}
for url in urls:
    df_dict[url] = read_data(urls[url])
    df_dict[url] = df_dict[url].drop(columns=['Unnamed: 3', 'Unnamed: 9'])
    df_dict[url] = df_dict[url].dropna()
    df_dict[url] = df_dict[url].rename(columns={"Price USD": url + '_USD'}, errors="raise")

In [105]:
df_dict['Basic'].head()

,Country,Basic,Currency,Basic_USD,# of TV Shows,# of Movies,Total Library Size,Price per Title,% Difference (Cost per Month) to Average,% Difference (Cost Per Month) to Most Expensive,% Difference (Cost Per Month) to Cheapest,% Difference (Cost per Title) to Average,% Difference (Cost per Title) to Most Expensive,% Difference (Cost per Title) to Cheapest
0,Turkey,26.99,TRY,1.97,2930.0,1709.0,4639.0,0.00043,123.76,146.85,0.00,118.52,163.41,0.00
1,India,199.00,INR,2.64,3718.0,2125.0,5843.0,0.00045,104.19,131.97,-28.87,114.56,161.37,-6.00
2,Argentina,379.00,ARS,3.74,3154.0,1606.0,4760.0,0.00079,76.63,110.04,-61.77,71.72,137.32,-59.43
3,Colombia,16900.00,COP,4.31,3156.0,1835.0,4991.0,0.00086,64.12,99.69,-74.39,63.26,132.11,-68.01
4,Brazil,25.90,BRL,4.61,3162.0,1810.0,4972.0,0.00093,58.00,94.53,-80.14,56.77,127.99,-74.24


In [106]:
df_dict['Basic'].columns

Index(['Country', 'Basic', 'Currency', 'Basic_USD', '# of TV Shows',
       '# of Movies', 'Total Library Size', 'Price per Title',
       '% Difference (Cost per Month) to Average',
       '% Difference (Cost Per Month) to Most Expensive',
       '% Difference (Cost Per Month) to Cheapest',
       '% Difference (Cost per Title) to Average',
       '% Difference (Cost per Title)  to Most Expensive',
       '% Difference (Cost per Title)  to Cheapest'],
      dtype='object')

In [135]:
#read Countries info
# https://statisticstimes.com/geography/countries-by-continents.php
# https://197travelstamps.com/countries-and-capitals-and-currencies/
# https://geo.koltyrin.ru/eng_countries_of_the_world.php
df_html_list = pd.read_html('https://statisticstimes.com/geography/countries-by-continents.php')
df_countries = df_html_list[0]
# df_countries = df_countries.rename(columns={"Country or Area": "Country"}, errors="raise")

3

In [137]:
df_countries

,No,Country,ISO-alpha3 Code,M49 Code,Region 1,Region 2,Continent
0,1,Afghanistan,AFG,4,Southern Asia,NaN,Asia
1,2,Åland Islands,ALA,248,Northern Europe,NaN,Europe
2,3,Albania,ALB,8,Southern Europe,NaN,Europe
3,4,Algeria,DZA,12,Northern Africa,NaN,Africa
4,5,American Samoa,ASM,16,Polynesia,NaN,Oceania
...,...,...,...,...,...,...,...
244,245,Wallis and Futuna Islands,WLF,876,Polynesia,NaN,Oceania
245,246,Western Sahara,ESH,732,Northern Africa,NaN,Africa
246,247,Yemen,YEM,887,Western Asia,NaN,Asia
247,248,Zambia,ZMB,894,Eastern Africa,Sub-Saharan Africa,Africa


In [161]:
df = df_dict['Basic'].merge(df_countries, left_on='Country', right_on='Country', how='left')
df = df.merge(df_dict['Standard'], left_on='Country', right_on='Country')
df = df.merge(df_dict['Premium'], left_on='Country', right_on='Country')

In [162]:
df.columns

Index(['Country', 'Basic', 'Currency_x', 'Basic_USD', '# of TV Shows_x',
       '# of Movies_x', 'Total Library Size_x', 'Price per Title_x',
       '% Difference (Cost per Month) to Average_x',
       '% Difference (Cost Per Month) to Most Expensive_x',
       '% Difference (Cost Per Month) to Cheapest_x',
       '% Difference (Cost per Title) to Average_x',
       '% Difference (Cost per Title)  to Most Expensive_x',
       '% Difference (Cost per Title)  to Cheapest_x', 'No', 'ISO-alpha3 Code',
       'M49 Code', 'Region 1', 'Region 2', 'Continent', 'Standard',
       'Currency_y', 'Standard_USD', '# of TV Shows_y', '# of Movies_y',
       'Total Library Size_y', 'Price per Title_y',
       '% Difference (Cost per Month) to Average_y',
       '% Difference (Cost Per Month) to Most Expensive_y',
       '% Difference (Cost Per Month) to Cheapest_y',
       '% Difference (Cost per Title) to Average_y',
       '% Difference (Cost per Title)  to Most Expensive_y',
       '% Difference (C

In [163]:
# 1. Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от валюты страны.  
df1 = df.groupby('Currency')[['Basic_USD', 'Standard_USD', 'Premium_USD']].mean()
df1.head()

,Basic_USD,Standard_USD,Premium_USD
Currency,,,
ARS,3.74,6.30,9.26
AUD,7.84,12.12,16.39
BRL,4.61,7.11,9.96
CAD,7.91,13.05,16.61
CHF,12.88,20.46,26.96


In [ ]:
# 2.Какая средняя цена подписки, (базовой, стандартной, премиум) в зависимости от континента

In [164]:
df2 = df.groupby('Continent')[['Basic_USD', 'Standard_USD', 'Premium_USD']].mean()
df2.head()

,Basic_USD,Standard_USD,Premium_USD
Continent,,,
Africa,6.260000,10.050000,12.580000
Asia,7.305556,10.450000,13.115556
Europe,9.331000,13.517667,17.894000
North America,7.900000,11.690000,14.964000
Oceania,8.320000,12.325000,16.665000


In [165]:
# 4. Какие топ-5 стран имеют самую большую доступную коллекцию сериалов? (Считаем, что сериалы это колонка '# of TV Shows')
df4 = df[['Country', '# of TV Shows']].sort_values(by=['# of TV Shows'], ascending=False, ignore_index=True)
df4.head(5)

,Country,# of TV Shows
0,Czech Republic,5234.0
1,Slovakia,5055.0
2,Bulgaria,4819.0
3,Hungary,4802.0
4,United Kingdom,4551.0


In [168]:
# 6. На каком месте будет Россия, если отсортировать страны по эффективности подписки (Cost Per Title)
# 26 место, т.к. считаем начиная с 0
df6 = df
df6['Cost Per Title'] = df['Basic_USD'] / df['Average Total Library Size']
df6 = df6[['Country', 'Cost Per Title']].sort_values(by=['Cost Per Title'], ascending=True, ignore_index=True)
df6[df6.Country == 'Russia']

,Country,Cost Per Title
25,Russia,0.001424


In [170]:
# 7. В какой стране лучше всего брать VPN для того, чтобы платить за Netflix меньше всего?
df7 = df[['Country', 'Basic_USD']].sort_values(by=['Basic_USD'], ascending=True, ignore_index=True)
df7.head(1)

,Country,Basic_USD
0,Turkey,1.97
